In [ ]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [ ]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("mapbox")

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year"""
    housing_per_year = sfo_data['housing_units']
    housing_units_per_year = housing_per_year.plot(kind='line', title='Housing Units Per Year')
    return housing_units_per_year

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    avg_gross_rent = sfo_data['gross_rent']
    average_gross_rent = avg_gross_rent.plot(kind='line', title='Average Gross Rent in San Francisco Per Year', figsize=(30,10))
    return average_gross_rent

def average_sales_price():
    """Average Sales Price Per Year."""
    avg_sales_price = sfo_data['sale_price_sqr_foot']
    average_sales_price = avg_sales_price.plot(kind='line', title='Average Sales Price Per Year')
    return average_sales_price

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    #avg_prices_by_nbh = sfo_data.groupby(['year','neighborhood'])['sale_price_sqr_foot'].mean()
    #avg_prices_by_nbh.plot(kind='line', title='Average Price by Neighborhood', figsize=(20,7))
    
    avg_price_neighborhood = sfo_data['sale_price_sqr_foot']
    average_price_by_neighborhood = avg_price_neighborhood.plot(kind='line', title='Average Sales Price Per Neighborhood', figsize=(20, 8))
    return average_price_by_neighborhood
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    mean_sale_price = sfo_data.reset_index()
    mean_sale_price.drop(labels='index', axis=1)
    top_ten = mean_sale_price.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False)
    top_ten_most_expensive = top_ten.hvplot.bar(x='neighborhood', y='sale_price_sqr_foot',
                       xlabel='San Francisco Neighboorhood', ylabel='$/sqft', 
                       rot=45, title='Top 10 Expensive Neighboorhoods in San Francisco, CA')
    return top_most_expensive_neighborhoods
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    mean_sale_price = sfo_data.reset_index()
    #mean_sale_price.drop(labels='index', axis=1)
    top_ten = mean_sale_price.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False)
    top_ten_px_data = top_ten[['sale_price_sqr_foot', 'housing_units', 'gross_rent']]
    fig1 = px.parallel_coordinates(top_ten_px_data, color='sale_price_sqr_foot', color_continuous_scale=px.colors.sequential.Inferno,
                        labels={"sale_price_sqr_foot": "$/sqft", "housing_units": "Housing Units", "gross_rent": "Gross Rent"})
    parallel_coordinates = fig1.show()
    return parallel_coordinates

def parallel_categories():
    """Parallel Categories Plot."""
    mean_sale_price = sfo_data.reset_index()
    top_ten = mean_sale_price.groupby('neighborhood').mean().sort_values('sale_price_sqr_foot', ascending=False)
    fig2 = px.parallel_categories(top_ten, color='sale_price_sqr_foot', color_continuous_scale=px.colors.sequential.Inferno,
                        labels={"neighboorhood": "neighborhood", "sale_price_sqr_foot": "$/sqft", "housing_units": "Housing Units", "gross_rent": "Gross Rent"})
    parallel_categories = fig2.show()
    return parallel_categories


def neighborhood_map():
    """Neighborhood Map"""
    sfo_data.reset_index(drop=True, inplace=True)
    df_neighborhood_locations.reset_index(drop=True, inplace=True)
    combined_df = pd.concat([sfo_data, df_neighborhood_locations], sort=False, axis=1)

    fig3 = px.scatter_mapbox(combined_df, lat='Lat', lon='Lon', color='gross_rent', zoom=10,
                         text = combined_df.index,
                         title='Gross Rent in San Francisco, CA')
    neighborhood_map = fig3.show()
    return neighborhood_map

In [ ]:
# Creating the panel dashboard
housing = pn.Column(
    "##San Francisco Housing Plots", housing_units_per_year, average_gross_rent, average_sales_price
)
neighborhood = pn.Column(
    "##San Francisco Housing by Neighborhood", average_price_by_neighborhood
)
#  ^top_most_expensive_neighborhoods


#  Because of a Recursion Error I was getting,
#  I failed to add top_most_expensive_neighborhoods column into my dashboard...
## RecursionError: maximum recursion depth exceeded while calling a Python object

map_features = pn.Column(
    "##San Francisco Housing Statistics and Map Features", parallel_coordinates, parallel_categories, neighborhood_map)

dashboard_tabs = pn.Tabs(
    ("Housing", housing),("Neigborhoods", neighborhood), ("PCs & MapBox", map_features))

In [ ]:
dashboard_tabs.servable()